In [1]:
class MyRegressor:
    def __init__(self) -> None:
        self.W = []
        self.X = []

    def get_parameters(self):
        return self.W
    
    def train(self, training_input_set, training_output_set):
        self.X = [[1] + line.copy() for line in training_input_set]
        XT = self.__transpose_matrix(self.X)
        P = self.__product(self.X,XT)

        P_Invers = self.__inversa(P)

        A = self.__product(P_Invers, XT)
        Y = [[el] for el in training_output_set]

        self.W = self.__product(A,Y)
        
    def __transpose_matrix(self, A):
        n, m = len(A), len(A[-1])
        XT = [[] for i in range(m)]
        for i in range(n):
            for j in range(m):
                XT[j].append(A[i][j])
        return XT

    def __product(self,A,B):
        nr_lines = len(A)
        nr_cols = len(B[-1])
        prod = []
        for i in range(nr_lines):
            prod.append([])
            for j in range(nr_cols):
                prod[i].append(sum([A[i][k] * B[k][j] for k in range(len(A[i]))]))

        return prod          

    def __elimLinCol(self, A, i:int, j:int):
        B  = [ A[k].copy() for k in range(len(A)) if k != i ]
        for line in B:
            line.pop(j)
        return B

    def __determinant(self, A):
        if len(A) == 1:
            return A[-1][-1]
        return sum(((-1) ** j) * A[0][j] * self.__determinant(self.__elimLinCol(A,0,j)) for j in range(len(A[0])))

    def __inversa(self, A):
        dA = self.__determinant(A)
        B = []
        nr_lines = len(A)
        nr_cols = len(A[-1])
        for i in range(nr_lines):
            B.append([])
            for j in range(nr_cols):
                B.append(self.__determinant(self.__elimLinCol(A,i,j))/dA)
        
        return B



In [2]:
regresor = MyRegressor()

In [4]:
import pandas as pd
import numpy as np

In [5]:
def getTrainingAndValidationSets(df_world_happiness):
    dataSize = df_world_happiness.shape[0]
    trainingIndexSet = np.random.choice(range(dataSize),size=int(0.8 * dataSize),replace=False)
    validationIndexSet = [i for i in range(dataSize) if i not in trainingIndexSet] 

    trainingInputSet = [df_world_happiness["Economy..GDP.per.Capita."].iloc[index] for index in trainingIndexSet ]
    trainingOutputSet = [df_world_happiness["Happiness.Score"].iloc[index] for index in trainingIndexSet ]

    validationInputSet = [df_world_happiness["Economy..GDP.per.Capita."].iloc[index] for index in validationIndexSet ]
    validationOutputSet = [df_world_happiness["Happiness.Score"].iloc[index] for index in validationIndexSet ]
    
    return trainingInputSet, trainingOutputSet, validationInputSet, validationOutputSet

In [6]:
world_happiness = pd.read_csv("datas/v1_world-happiness-report-2017.csv",delimiter=',', header='infer')

In [7]:
training_input_set, training_output_set, validation_input_set, validation_output_set = getTrainingAndValidationSets(world_happiness)

training_input_set = [[el] for el in training_input_set]
regresor.train(training_input_set, training_output_set)